In [1]:
from ipywidgets import interact, interactive
from IPython.display import display
import ipywidgets as widgets
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os
%matplotlib inline

In [2]:
dataPath = './BRATS2015_Training'

## 数据

### 先读个图出来看看

#### 选一个文件

In [3]:
def generatePathDictionary(startPath):
    if os.path.isdir(startPath):
        return {
            currentPath : generatePathDictionary(startPath  + '/' + currentPath) \
            for currentPath in os.listdir(startPath)
        }
    elif os.path.isfile(startPath):
#         print startPath
        return startPath
    else:
        print startPath
        raise ValueError

In [4]:
dic = generatePathDictionary(dataPath)

firstList = []
secondList = []
thirdList = []
lastList = []

def firstFunction(cfirst):
    firstList.append(cfirst)
    secondRange.options = dic[cfirst].keys()
    
def secondFunction(csecond):
    secondList.append(csecond)
    thirdRange.options = dic[firstList[-1]][csecond].keys()

def thirdFunction(cthird):
    thirdList.append(cthird)
    fouthRange.options = dic[firstList[-1]][secondList[-1]][cthird].keys()
    

    
def lastFunction(clast):
    lastList.append(
        dic[firstList[-1]][secondList[-1]][thirdList[-1]][clast])
    print lastList[-1]
    
firstRange = widgets.Select(options = dic.keys())
first = firstRange.value

secondRange = widgets.Select(options = dic[first].keys())
second = secondRange.value

thirdRange = widgets.Select(options = dic[first][second].keys())
third = thirdRange.value

fouthRange = widgets.Select(options = dic[first][second][third].keys())
fouth = fouthRange.value

display(interactive(firstFunction, cfirst = firstRange))
display(interactive(secondFunction, csecond = secondRange))
display(interactive(thirdFunction, cthird = thirdRange))
display(interactive(lastFunction, clast = fouthRange))

# interact(firstFunction, cfirst = firstRange)
# interact(secondFunction, csecond = secondRange)
# interact(thirdFunction, cthird = thirdRange)
# interact(lastFunction, clast = fouthRange)

./BRATS2015_Training


ValueError: 

**目前代码还不完善，需要手动将上面输出的路径拷贝到变量中**

In [5]:
chooseFilePath = '''./BRATS2015_Training/LGG/brats_tcia_pat152_0001/VSD.Brain.XX.O.MR_T1c.40837/VSD.Brain.XX.O.MR_T1c.40837.mha'''

#### 绘制

In [2]:
import ipyvolume.pylab as p3
import ipyvolume
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [7]:
def show3D(filePath):
    image = sitk.ReadImage(filePath)
    img = sitk.GetArrayFromImage(image)
    
    display(ipyvolume.volume.quickvolshow(img))
    

In [3]:
def showSlice(filePath):
    image = sitk.ReadImage(filePath)
    img = sitk.GetArrayFromImage(image)
    zSlice = img[1, :, :]
    pl = figure(
        plot_width=400, 
        plot_height=400, 
        x_range=(0, 10), 
        y_range=(0, 10))
    slicer = pl.image(
        image = [zSlice], 
        x=[0], 
        y=[0], 
        dw=[10], 
        dh=[10])
    
    def update(z):
        zSlicer = img[z, :, :]
        slicer.data_source.data['image'] = [zSlicer]
        push_notebook()
        
    show(pl, notebook_handle = True)
    play = widgets.Play(
        value=50,
        min=0,
        max=154,
        step=1,
        description="Press play",
        disabled=False)
    slider = widgets.IntSlider()
    widgets.jslink((play, 'value'), (slider, 'value'))
    interact(update, z = play)
    display(slider)

##### 切片图

In [4]:
def showSlice(img):
    zSlice = img[1, :, :]
    pl = figure(
        plot_width=400, 
        plot_height=400, 
        x_range=(0, 10), 
        y_range=(0, 10))
    slicer = pl.image(
        image = [zSlice], 
        x=[0], 
        y=[0], 
        dw=[10], 
        dh=[10])
    
    def update(z):
        zSlicer = img[z, :, :]
        slicer.data_source.data['image'] = [zSlicer]
        push_notebook()
        
    show(pl, notebook_handle = True)
    play = widgets.Play(
        value=50,
        min=0,
        max=154,
        step=1,
        description="Press play",
        disabled=False)
    slider = widgets.IntSlider()
    widgets.jslink((play, 'value'), (slider, 'value'))
    interact(update, z = play)
    display(slider)

In [3]:
filePath = '/home/mountain/Desktop/BrainNets/output/dilated3DNetOutputForNii/Test17-07-17_00:08:43/Brats17_CBICA_APM_1/result.nii.gz'

In [62]:
maskPath = '/home/mountain/Desktop/BrainNets/output/dilated3DNetOutputForNii/17-07-12_21:07:22/Brats17_2013_22_1/resultMask.npy'

In [4]:
labelPath = '/home/mountain/Desktop/BrainNets/output/dilated3DNetOutputForNii/Test17-07-17_00:08:43/Brats17_CBICA_APM_1/crfResult.nii.gz'

In [5]:
import nibabel as nib

In [6]:
labelArray = nib.load(labelPath)

In [7]:
labelArray = labelArray.get_data()

In [65]:
labelArray = np.load(labelPath)

In [8]:
imageArray = nib.load(filePath)

In [9]:
imageArray = imageArray.get_data()

In [9]:
maskArray = np.load(maskPath)

In [66]:
imageArray = np.load(filePath)

In [10]:
def showNiiSlice(img):
    zSlice = img[:, :, 1]
    pl = figure(
        plot_width=400, 
        plot_height=400, 
        x_range=(0, 10), 
        y_range=(0, 10))
    slicer = pl.image(
        image = [zSlice], 
        x=[0], 
        y=[0], 
        dw=[10], 
        dh=[10])
    
    def update(z):
        zSlicer = img[:, :, z]
        slicer.data_source.data['image'] = [zSlicer]
        push_notebook()
        
    show(pl, notebook_handle = True)
    play = widgets.Play(
        value=50,
        min=0,
        max=154,
        step=1,
        description="Press play",
        disabled=False)
    slider = widgets.IntSlider()
    widgets.jslink((play, 'value'), (slider, 'value'))
    interact(update, z = play)
    display(slider)

In [11]:
showNiiSlice(imageArray)

In [12]:
showNiiSlice(labelArray)

In [13]:
import utils.general as ge

In [14]:
ge.numpyArrayCounter(imageArray)

array([[      0, 8476194],
       [      1,  187070],
       [      2,  221262],
       [      4,   43474]])

In [15]:
ge.numpyArrayCounter(labelArray)

array([[      0, 8913790],
       [      1,   11048],
       [      3,    3162]])

In [16]:
imageArray[125][125]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

In [17]:
labelArray[125][125]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

In [12]:
imageArray[120][120] != labelArray[120][120]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False,  True,  True,
        True,  True,  True, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [18]:
import utils.metrics as ms

In [20]:
ms.voxleWiseMetrics(imageArray, labelArray, [1,2])

(0.027955553436024606, 0.53059377262853002, 0.95486462190219257)

In [21]:
display(ipyvolume.volume.quickvolshow(imageArray))

/home/mountain/anaconda3/envs/3dcrf/lib/python2.7/site-packages/ipyvolume/serialize.py:29: RuntimeWarning: invalid value encountered in divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


In [22]:
display(ipyvolume.volume.quickvolshow(labelArray))

In [10]:
display(ipyvolume.volume.quickvolshow(maskArray))

In [9]:
showSlice(chooseFilePath)

RuntimeError: Exception thrown in SimpleITK ReadImage: /opt/miniconda2/conda-bld/simpleitk_1491578024069/work/Code/IO/src/sitkImageReaderBase.cxx:74:
sitk::ERROR: The file "./BRATS2015_Training/LGG/brats_tcia_pat152_0001/VSD.Brain.XX.O.MR_T1c.40837/VSD.Brain.XX.O.MR_T1c.40837.mha" does not exist.

##### 3D交互式绘图

In [10]:
show3D(chooseFilePath)

/home/mountain/anaconda3/envs/biomediclasagne/lib/python2.7/site-packages/ipyvolume/serialize.py:29: RuntimeWarning: invalid value encountered in divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


In [17]:
probs = np.random.randn(5, 69, 51).astype(np.float32)
probs /= probs.sum(axis=0, keepdims=True)

In [18]:
probs.shape

(5, 69, 51)

In [20]:
probs.shape[0]

5

In [19]:
probs

array([[[  1.31222814e-01,   4.32115984e+00,   8.57622623e-01, ...,
          -2.12118849e-02,  -9.04378388e-03,   9.97004583e-02],
        [  2.72405148e-01,  -3.27040482e+01,  -7.43244350e-01, ...,
           3.11578736e-02,   1.40256858e+00,  -4.33338970e-01],
        [  1.70033142e-01,   3.50571513e-01,   4.90989208e-01, ...,
           3.25156212e-01,   8.08748484e-01,   4.41498570e-02],
        ..., 
        [  3.09758127e-01,  -1.74558133e-01,  -6.59447849e-01, ...,
          -1.23846954e+02,  -9.30796683e-01,   1.63205636e+00],
        [ -2.59871274e-01,   1.80449257e+01,   1.20163232e-01, ...,
           2.00068593e-01,   4.04199094e-01,   1.29495883e+00],
        [ -1.15591545e+01,   7.02330530e-01,  -1.59039760e+00, ...,
          -1.42019200e+00,   3.03341895e-01,   1.53172970e-01]],

       [[  7.50291824e-01,   8.98784161e+00,   1.06881762e+00, ...,
           2.50131279e-01,   4.63324517e-01,   6.56866491e-01],
        [  1.67359102e+00,   9.25334778e+01,   6.64896607e-0

In [57]:
a = np.zeros((3, 4, 5))

In [58]:
a

array([[[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]]])

In [59]:
a[:,1:3, 3:5] = b

In [60]:
a

array([[[  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   1.],
        [  0.,   0.,   0.,   2.,   3.],
        [  0.,   0.,   0.,   0.,   0.]],

       [[  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   4.,   5.],
        [  0.,   0.,   0.,   6.,   7.],
        [  0.,   0.,   0.,   0.,   0.]],

       [[  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   8.,   9.],
        [  0.,   0.,   0.,  10.,  11.],
        [  0.,   0.,   0.,   0.,   0.]]])

In [55]:
b = np.arange(12).reshape((3, 2, 2))

In [56]:
b

array([[[ 0,  1],
        [ 2,  3]],

       [[ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11]]])

In [23]:
a.shape

(3, 4, 5)

In [24]:
a[0]

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

In [31]:
b = np.arange(3 * 4 * 2 * 2* 2).reshape((3,4,2,2,2))

In [32]:
b

array([[[[[ 0,  1],
          [ 2,  3]],

         [[ 4,  5],
          [ 6,  7]]],


        [[[ 8,  9],
          [10, 11]],

         [[12, 13],
          [14, 15]]],


        [[[16, 17],
          [18, 19]],

         [[20, 21],
          [22, 23]]],


        [[[24, 25],
          [26, 27]],

         [[28, 29],
          [30, 31]]]],



       [[[[32, 33],
          [34, 35]],

         [[36, 37],
          [38, 39]]],


        [[[40, 41],
          [42, 43]],

         [[44, 45],
          [46, 47]]],


        [[[48, 49],
          [50, 51]],

         [[52, 53],
          [54, 55]]],


        [[[56, 57],
          [58, 59]],

         [[60, 61],
          [62, 63]]]],



       [[[[64, 65],
          [66, 67]],

         [[68, 69],
          [70, 71]]],


        [[[72, 73],
          [74, 75]],

         [[76, 77],
          [78, 79]]],


        [[[80, 81],
          [82, 83]],

         [[84, 85],
          [86, 87]]],


        [[[88, 89],
          [90, 91]],

        

In [33]:
bShape = b.shape

In [34]:
c = np.transpose(b, (0, 2, 3, 4, 1))

In [35]:
c

array([[[[[ 0,  8, 16, 24],
          [ 1,  9, 17, 25]],

         [[ 2, 10, 18, 26],
          [ 3, 11, 19, 27]]],


        [[[ 4, 12, 20, 28],
          [ 5, 13, 21, 29]],

         [[ 6, 14, 22, 30],
          [ 7, 15, 23, 31]]]],



       [[[[32, 40, 48, 56],
          [33, 41, 49, 57]],

         [[34, 42, 50, 58],
          [35, 43, 51, 59]]],


        [[[36, 44, 52, 60],
          [37, 45, 53, 61]],

         [[38, 46, 54, 62],
          [39, 47, 55, 63]]]],



       [[[[64, 72, 80, 88],
          [65, 73, 81, 89]],

         [[66, 74, 82, 90],
          [67, 75, 83, 91]]],


        [[[68, 76, 84, 92],
          [69, 77, 85, 93]],

         [[70, 78, 86, 94],
          [71, 79, 87, 95]]]]])

In [36]:
c.reshape((bShape[0] * bShape[2]*bShape[3]*bShape[4], bShape[1]))

array([[ 0,  8, 16, 24],
       [ 1,  9, 17, 25],
       [ 2, 10, 18, 26],
       [ 3, 11, 19, 27],
       [ 4, 12, 20, 28],
       [ 5, 13, 21, 29],
       [ 6, 14, 22, 30],
       [ 7, 15, 23, 31],
       [32, 40, 48, 56],
       [33, 41, 49, 57],
       [34, 42, 50, 58],
       [35, 43, 51, 59],
       [36, 44, 52, 60],
       [37, 45, 53, 61],
       [38, 46, 54, 62],
       [39, 47, 55, 63],
       [64, 72, 80, 88],
       [65, 73, 81, 89],
       [66, 74, 82, 90],
       [67, 75, 83, 91],
       [68, 76, 84, 92],
       [69, 77, 85, 93],
       [70, 78, 86, 94],
       [71, 79, 87, 95]])

In [37]:
c.reshape((bShape[0] ,bShape[2], bShape[3], bShape[4], bShape[1]))

array([[[[[ 0,  8, 16, 24],
          [ 1,  9, 17, 25]],

         [[ 2, 10, 18, 26],
          [ 3, 11, 19, 27]]],


        [[[ 4, 12, 20, 28],
          [ 5, 13, 21, 29]],

         [[ 6, 14, 22, 30],
          [ 7, 15, 23, 31]]]],



       [[[[32, 40, 48, 56],
          [33, 41, 49, 57]],

         [[34, 42, 50, 58],
          [35, 43, 51, 59]]],


        [[[36, 44, 52, 60],
          [37, 45, 53, 61]],

         [[38, 46, 54, 62],
          [39, 47, 55, 63]]]],



       [[[[64, 72, 80, 88],
          [65, 73, 81, 89]],

         [[66, 74, 82, 90],
          [67, 75, 83, 91]]],


        [[[68, 76, 84, 92],
          [69, 77, 85, 93]],

         [[70, 78, 86, 94],
          [71, 79, 87, 95]]]]])

In [40]:
c

array([[[[[ 0,  8, 16, 24],
          [ 1,  9, 17, 25]],

         [[ 2, 10, 18, 26],
          [ 3, 11, 19, 27]]],


        [[[ 4, 12, 20, 28],
          [ 5, 13, 21, 29]],

         [[ 6, 14, 22, 30],
          [ 7, 15, 23, 31]]]],



       [[[[32, 40, 48, 56],
          [33, 41, 49, 57]],

         [[34, 42, 50, 58],
          [35, 43, 51, 59]]],


        [[[36, 44, 52, 60],
          [37, 45, 53, 61]],

         [[38, 46, 54, 62],
          [39, 47, 55, 63]]]],



       [[[[64, 72, 80, 88],
          [65, 73, 81, 89]],

         [[66, 74, 82, 90],
          [67, 75, 83, 91]]],


        [[[68, 76, 84, 92],
          [69, 77, 85, 93]],

         [[70, 78, 86, 94],
          [71, 79, 87, 95]]]]])

In [38]:
d = np.transpose(c, (0, 4, 1, 2, 3))

In [39]:
np.all(d == b)

True

In [44]:
(1,2,3) + (4,)

(1, 2, 3, 4)